In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # Windows
# matplotlib.rcParams['font.family'] = 'AppleGothic' # Mac
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['axes.unicode_minus'] = False

In [357]:
df_seoul = pd.read_csv('test/0320_test/ta_seoul.csv', encoding='cp949')
df_daegu = pd.read_csv('test/0320_test/ta_daegu.csv', encoding='cp949')

In [358]:
df_seoul = pd.DataFrame(df_seoul)
df_seoul.columns = ['날짜', '지점', '서울평균', '서울최저', '서울최고']
df_daegu = pd.DataFrame(df_daegu)
df_daegu.columns = ['날짜', '지점', '대구평균', '대구최저', '대구최고']

In [359]:
df_seoul = df_seoul.fillna(np.round(df_seoul.mean(), 2))
df_daegu = df_daegu.fillna(np.round(df_daegu.mean(), 2))
print(df_seoul.mean().value_counts())
print(df_daegu.mean().value_counts())

108.000000    1
11.737481     1
7.445352      1
16.743098     1
dtype: int64
143.000000    1
13.240024     1
8.493513      1
18.868982     1
dtype: int64


In [360]:
print(df_seoul.isna().sum())
print(df_daegu.isna().sum())

날짜      0
지점      0
서울평균    0
서울최저    0
서울최고    0
dtype: int64
날짜      0
지점      0
대구평균    0
대구최저    0
대구최고    0
dtype: int64


In [361]:
df_seoul['날짜'] = pd.to_datetime(df_seoul['날짜'])
df_daegu['날짜'] = pd.to_datetime(df_daegu['날짜'])
print(df_seoul['날짜'].head(3))
print(df_daegu['날짜'].head(3))

0   1907-10-01
1   1907-10-02
2   1907-10-03
Name: 날짜, dtype: datetime64[ns]
0   1907-01-31
1   1907-02-01
2   1907-02-02
Name: 날짜, dtype: datetime64[ns]


### 1. 기상 관측 이래, 서울의 최고 기온이 가장 높았던 날은 언제였고, 몇도인가요?

In [362]:
res = df_seoul[['날짜', '서울최고']][df_seoul['서울최고'] == df_seoul['서울최고'].max()]
res

,날짜,서울최고
40051,2018-08-01,39.6


### 2. 역사적으로 일교차가 가장 큰 날짜는 몇년 몇월 몇일 인가요?

In [363]:
df_seoul['서울일교차'] = df_seoul['서울최고'] - df_seoul['서울최저']
res = df_seoul['날짜'][df_seoul['서울일교차'] == df_seoul['서울일교차'].max()].to_frame()
res

,날짜
12619,1942-04-19


In [364]:
df_daegu['대구일교차'] = df_daegu['대구최고'] - df_daegu['대구최저']
res = df_daegu['날짜'][df_daegu['대구일교차'] == df_daegu['대구일교차'].max()].to_frame()
res

,날짜
2255,1913-04-04


### 3. 1년 중 평균적으로 일교차가 가장 큰 날짜는 몇월 몇일 인가요?

In [365]:
df_seoul['서울_월'] = df_seoul['날짜'].dt.month.astype('str').map(lambda x : "0"+x if len(x) == 1 else x)
df_seoul['서울_요일'] = df_seoul['날짜'].dt.day.astype('str').map(lambda x : "0"+x if len(x) == 1 else x)
df_seoul['서울_월일'] = df_seoul['서울_월'] + "-" + df_seoul['서울_요일']
res = df_seoul[['서울_월일', '서울일교차']].groupby('서울_월일', as_index=False)['서울일교차'].mean()
res = res[['서울_월일', '서울일교차']][res['서울일교차'] == res['서울일교차'].max()]
res['서울일교차'] = np.round(res['서울일교차'], 2)
res.set_index('서울_월일')

,서울일교차
서울_월일,
04-29,11.88


In [366]:
df_daegu['대구_월'] = df_daegu['날짜'].dt.month.astype('str').map(lambda x : "0"+x if len(x) == 1 else x)
df_daegu['대구_요일'] = df_daegu['날짜'].dt.day.astype('str').map(lambda x : "0"+x if len(x) == 1 else x)
df_daegu['대구_월일'] = df_daegu['대구_월'] + "-" + df_daegu['대구_요일']
res = df_daegu[['대구_월일', '대구일교차']].groupby('대구_월일', as_index=False)['대구일교차'].mean()
res = res[['대구_월일', '대구일교차']][res['대구일교차'] == res['대구일교차'].max()]
res['대구일교차'] = np.round(res['대구일교차'], 2)
res.set_index('대구_월일')

,대구일교차
대구_월일,
05-22,12.86


### 4. 가장 덥다고 알려진 대구보다 서울이 더 더운날이 가장 많은 연도는 언제인가요?

In [367]:
df_daegu.drop(['대구_월', '대구_요일', '대구_월일'], axis=1)
df_seoul.drop(['서울_월', '서울_요일', '서울_월일'], axis=1)

,날짜,지점,서울평균,서울최저,서울최고,서울일교차
0,1907-10-01,108,13.5,7.9,20.7,12.8
1,1907-10-02,108,16.2,7.9,22.0,14.1
2,1907-10-03,108,16.2,13.1,21.3,8.2
3,1907-10-04,108,16.5,11.2,22.0,10.8
4,1907-10-05,108,17.6,10.9,25.4,14.5
...,...,...,...,...,...,...
41374,2022-03-16,108,10.3,3.6,16.8,13.2
41375,2022-03-17,108,10.7,7.8,15.1,7.3
41376,2022-03-18,108,7.5,5.5,9.6,4.1
41377,2022-03-19,108,3.4,0.9,6.1,5.2


In [368]:
df = pd.merge(df_seoul, df_daegu, on='날짜', how='inner')
df.head(5)

,날짜,지점_x,서울평균,서울최저,서울최고,서울일교차,서울_월,서울_요일,서울_월일,지점_y,대구평균,대구최저,대구최고,대구일교차,대구_월,대구_요일,대구_월일
0,1907-10-01,108,13.5,7.9,20.7,12.8,10,01,10-01,143,13.24,8.49,18.87,10.38,10,01,10-01
1,1907-10-02,108,16.2,7.9,22.0,14.1,10,02,10-02,143,13.24,8.49,18.87,10.38,10,02,10-02
2,1907-10-03,108,16.2,13.1,21.3,8.2,10,03,10-03,143,13.24,8.49,18.87,10.38,10,03,10-03
3,1907-10-04,108,16.5,11.2,22.0,10.8,10,04,10-04,143,13.24,8.49,18.87,10.38,10,04,10-04
4,1907-10-05,108,17.6,10.9,25.4,14.5,10,05,10-05,143,13.24,8.49,18.87,10.38,10,05,10-05


In [373]:
res = df[['날짜']][df['서울최고'] > df['대구최고']]
res = res['날짜'].dt.year.value_counts()
res = res.reset_index()
res.columns = ['연도', '일']
res[:1]

,연도,일
0,1908,166
